# Testing notebook
This notebook should, hopefully, speed up testing. It is broken up by section for independent usage. 

## Generate report

In [1]:
import pandas as pd

import datasources
import investigator
import reporter

patient = {}
patient_id = 'patient_id'
patient[patient_id] = 'example'
patient['oncotree_code'] = 'SKCM'
patient['oncotree_term'] = 'Cutaneous Melanoma'
patient['stage'] = 'Metastatic'
patient['description'] = ''
patient['date'] = ''
patient['purity'] = 100
patient['ploidy'] = 2.0
patient['microsatellite_status'] = 'MSI'

actionable = pd.read_csv('output/example.actionable.txt', sep='\t')
matchmaking = pd.read_csv('output/example.matchmaker.txt', sep='\t')
somatic = pd.read_csv('output/example.somatic.scored.txt', sep='\t')
dbs_preclinical = datasources.Preclinical.import_dbs()

efficacy_dictionary = investigator.SensitivityDictionary.create(dbs_preclinical, actionable, patient[patient_id])
efficacy_summary = investigator.SummaryDataFrame.create(efficacy_dictionary, actionable, patient[patient_id])

report_dictionary = reporter.Reporter.generate_dictionary(somatic, patient)
version_dictionary = reporter.Reporter.generate_version_dictionary()

matchmaker_on_boolean=True

dictionary = dbs_preclinical['dictionary']

evidence_map = {
    'FDA-Approved': 5.0, 'Guideline': 4.0, 'Clinical trial': 3.0,
    'Clinical evidence': 2.0, 'Preclinical': 1.0, 'Inferential': 0.0}

bin_map = {
        'Biologically Relevant': 1,
        'Investigate Actionability - Low': 2,
        'Investigate Actionability - High': 3,
        'Putatively Actionable': 4}

for column in ['sensitive', 'resistance', 'prognostic']:
    actionable[f'{column}_predictive_implication_map'] = actionable[f'{column}_predictive_implication'].replace(evidence_map)
    actionable[f'{column}_score_bin_map'] = actionable[f'{column}_score_bin'].replace(bin_map)
actionable['score_bin_map'] = actionable['score_bin'].replace(bin_map)

In [4]:
reporter.Reporter.generate_report(actionable, report_dictionary, version_dictionary,
                                  efficacy_dictionary, efficacy_summary,
                                  matchmaker_on_boolean, matchmaking, dictionary)

!mv build/index.html output/example.report.html